# Example on Downloading Earthquake Waveform Data from IRIS

This notebook demonstrates the process of downloading earthquake waveform data for the purpose of receiver function analysis using [rf](https://github.com/trichter/rf) package. It retrieves 3-component data for eartquakes in the event catalog.

We first import all the necessary modules 

In [ ]:
#%% Download data from IRIS 
import os 
import scipy
import matplotlib.pyplot as plt
import numpy as np
from obspy import read_inventory, read_events, UTCDateTime as UTC
from obspy.clients.fdsn import Client
from rf import read_rf, RFStream
from rf  import iter_event_data, IterMultipleComponents
from tqdm import tqdm
from os.path import exists
from os import mkdir
from rfsed.util import save_Eq_data, save_IRIS_waveform

Define the Station and Data Center Parameters as well as the file to save the data

In [ ]:
#------------------------------------------#
savedir=save_IRIS_waveform()
invfile = savedir + '00_eq_stations.xml'
catfile = savedir + '00_eq_events.xml'
datafile = savedir + '00_eq_data.h5'
#------------------------------------------#
# Input Examples
staname = 'OPLO'
Network = 'NL'
StartDate = '2020-01-01'
EndDate = '2020-08-31'
StationClient='ORFEUS'
EventClient='GFZ'
DataClient='ORFEUS'

Same as the previous cell: Define the Station and Data Center Parameters as well as the file to save the data

You can remove the comment in this cell  to make the parameter input flexible at an input prompt.

In [ ]:
# #------------------------------------------#
# savedir=save_IRIS_waveform()
# invfile = savedir + '00_eq_stations.xml'
# catfile = savedir + '00_eq_events.xml'
# datafile = savedir + '00_eq_data.h5'
# #------------------------------------------#
# stname = input("Input Station Name:")
# staname= str(stname)
# Network = str(input("Input Network Name:"))
# StartDate = str(input("Input Start Date (YYYY-MM-DD):"))
# EndDate = str(input("Input End Date (YYYY-MM-DD):"))
# StationClient=str(input("Input Station Client e.g. IRIS, ORFEUS, GFZ:"))
# EventClient=str(input("Input Event Client e.g. IRIS, ORFEUS, GFZ:"))
# DataClient=str(input("Input Data Client e.g. IRIS, ORFEUS, GFZ:"))

Next is the downloading of the Station Inventory file from the appropriate data center.

In [ ]:
#------------------------------------------#
# Download station inventory
minlat=50.3; maxlat=53.8  #South-North
minlong=3.0; maxlong=7.7 #West-East
if not os.path.exists(invfile):
    client = Client(StationClient)
    inventory = client.get_stations(network=Network, channel='BH?', level='channel', station=staname,
                                    minlatitude = minlat, maxlatitude = maxlat, 
                                    minlongitude = minlong, maxlongitude = maxlong)
    inventory.write(invfile, 'STATIONXML')
inventory = read_inventory(invfile)
# inventory.plot(label=False)
# fig = inventory.plot('local')
#------------------------------------------#

Next, the catalog of earthquake events within the period of interest and of defined source charateristics is downloaded from appropriate data center.

In [ ]:
# Download event catalog
stacomp= Network + '.' + staname + '..BHZ'
coords = inventory.get_coordinates(stacomp)
lonlat = (coords['longitude'], coords['latitude'])
long=lonlat[0]
lat=lonlat[1]
if not os.path.exists(catfile):
    client = Client(EventClient)
    kwargs = {'starttime': UTC(StartDate), 'endtime': UTC(EndDate),
              'latitude': lonlat[1], 'longitude': lonlat[0],
              'minradius': 30, 'maxradius': 95,
              'minmagnitude': 6, 'maxmagnitude': 10}
    catalog = client.get_events(**kwargs)

    catalog.write(catfile, 'QUAKEML')
catalog = read_events(catfile)
# fig = catalog.plot(label=None) 

The waveform data of the events in the catalog are then downloaded from the appropriate Data Center. The [rf](https://github.com/trichter/rf) preset of length of waveform downloaded for P-receiver function is 50 s before the direct-P wave and 150 s after its arrival.

In [ ]:
#------------------------------------------#
# Get the data 
if not os.path.exists(datafile):
    client = Client(DataClient)
    stream = RFStream()
    with tqdm() as pbar:
        for s in iter_event_data(catalog, inventory, client.get_waveforms, pbar=pbar):
            stream.extend(s)
    stream.write(datafile, 'H5')  
#%%
eqdata = read_rf(datafile)
for i in eqdata: 
    print(i)
print("Number of Events:", len(eqdata)/3, 'events')
# %%
